# First attempt to use datalab to connect to google BigQuery and do some analysis

Here's what I'm trying to do. I'm going to get on the Google BigQuery UI, test a couple of queries to see if it returns the data in the way I need it to be. Once that's finalized, I'm going to copy that query and assign it to a string variable in python. 

Google has a nice library to interface with Google BigQuery by passing the query as a string. I can then turn the result into a pandas dataframe for easy manipulation.

Let's run a test on any query we can think of and see if we can turn it into a pandas dataframe

In [20]:
query = """select * from `bigquery-public-data.stackoverflow.posts_answers`
limit 100"""

import google.datalab.bigquery as bq

dataframe = bq.Query(query).execute().result().to_dataframe()
dataframe.head()

,id,body,comment_count,community_owned_date,creation_date,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags
0,37951034,<p>You can use either <code>File.GetLastWriteT...,0,None,2016-06-21 17:38:02.057,2016-06-21 17:38:02.057,NaT,,NaN,,634824.0,13600907,2,0,
1,21399644,<p>you can check the url of the page you are o...,0,None,2014-01-28 07:44:43.613,2014-01-28 07:44:43.613,NaT,,NaN,,2542215.0,21399459,2,0,
2,39282103,<p>I've gone through your example and fixed th...,0,None,2016-09-01 23:05:33.863,2016-09-01 23:05:33.863,NaT,,NaN,,5667951.0,39281603,2,0,
3,35641348,"<p>This is what I figured out, using friendly ...",0,None,2016-02-26 00:51:06.067,2016-02-26 00:51:06.067,NaT,,NaN,,302760.0,35641209,2,0,
4,46295684,<p>You can replace the first DOM text child as...,0,None,2017-09-19 08:37:01.153,2017-09-19 17:06:29.723,2017-09-19 17:06:29.723,,230710.0,,230710.0,46295113,2,0,


### So it worked! Woohoo! 

Now we need to work on a query that can get data in a form that is useful. 

My goal is to predict whether a certain answer is the correct answer for a specific post.
I'm going to join the answers to the questions, and then subsequently to the profile of the users that answered. 

I hope this query doesn't take away all my gcloud trial money!

Let's create a giant query that will get info on the question, the answer and aggregate data on the user.

This was tested in MySQL, then adapted to standard SQL. It's a little annoying that the BigQuery UI uses MySQL but the bigquery library in datalab can only accept standard SQL.
<br><br>
## What kind of features am I trying to get?
1. Question Title, Body, Score, Comment Count, Tags
2. Answer Body, Comment Count, Score
3. User Number of Badges, Number of Comments Made, Number of Answers Posted, Number of Answers that had code in them, Reputation, UpVotes, DownVotes, ProfileViews, All Tags answered by user that made a 'good contribution'

In [16]:
query = """
select 
Questions.title as Questiontitle , 
Questions.body as QuestionBody,
Questions.comment_count QuestionCommentCount,
Questions.score as QuestionScore,
Questions.tags as QuestionTags,
Answers.body as AnswerBody,
Answers.comment_count as AnswerCommentCount,
Answers.score as AnswerScore,
UserDetails.NumberOfBadges,
UserDetails.NumberOfCommentsMade,
UserDetails.NumberOfAnswers,
UserDetails.CodedAnswers,
UserDetails.Reputation,
UserDetails.UpVotes,
UserDetails.DownVotes,
UserDetails.ProfileViews,
UserDetails.HasWebsite,
UserDetails.AllTagsAnswered,
(Answers.score>0) as IsItUseful, 
(case when Questions.accepted_answer_id is not null then Questions.accepted_answer_id=Answers.id else False end) as IsItAccepted,
(Answers.score>0 or (case when Questions.accepted_answer_id is not null then Questions.accepted_answer_id=Answers.id else False end)) as UsefulOrAccepted
from
`bigquery-public-data.stackoverflow.posts_questions` as Questions
join
`bigquery-public-data.stackoverflow.posts_answers` as Answers
on Questions.id = Answers.parent_id
join

(
  select Badges.user_id as User, Badges.NumberBadges as NumberOfBadges, Comments.NumberComments as NumberOfCommentsMade,
  Answers.NumberAnswered as NumberOfAnswers, Answers2.NumberAnsweredCode as CodedAnswers, Users.reputation as Reputation, Users.up_votes as UpVotes, Users.down_votes as DownVotes,
  Users.views as ProfileViews, Users.HasWebsite as HasWebsite, AllTagsAnswered.TagsAnswered as AllTagsAnswered
  from
  (
    select user_id, count(id) as NumberBadges from `bigquery-public-data.stackoverflow.badges` group by user_id
  ) as Badges 
  join 
  (
    select user_id, count(id) as NumberComments from `bigquery-public-data.stackoverflow.comments` group by user_id 
  ) as Comments
  on Badges.user_id = Comments.user_id
  join
  (
    select owner_user_id, count(id) as NumberAnswered from `bigquery-public-data.stackoverflow.posts_answers` group by owner_user_id 
  ) as Answers
  on Badges.user_id = Answers.owner_user_id
  join
  (
    select owner_user_id, countif(regexp_contains(body,'<code>')) as NumberAnsweredCode from `bigquery-public-data.stackoverflow.posts_answers` group by owner_user_id 
  ) as Answers2
  on Badges.user_id = Answers2.owner_user_id
  join 
  (
    select id, reputation, up_votes, down_votes, views, (website_url != "") as HasWebsite
    from `bigquery-public-data.stackoverflow.users`
  ) as Users
  on Badges.user_id = Users.id
  join
  (
    select TagsAnswered.user_id as user_id, array_agg(TagsAnswered.tags) as TagsAnswered
    from
    (
      select
      A.owner_user_id as user_id, B.id as post_id, B.tags as tags
      from 
      `bigquery-public-data.stackoverflow.posts_answers` as A
      join
      (
        select id, tags from `bigquery-public-data.stackoverflow.posts_questions`
      ) as B
      on A.parent_id = B.id
      where A.score >= 1
    ) as TagsAnswered
    group by user_id
  ) as AllTagsAnswered
  on AllTagsAnswered.user_id = Badges.user_id
) as UserDetails
on UserDetails.User = Answers.owner_user_id 
where Questions.score >= 0 """ 

## Phew! that was a lot of SQL! 
  

Let's execute and store the data into a dataframe. Let's start by limiting the reponses to a hundred so that I can look at my final table for errors. 

Check it out.

I can just append a limit term to the query above so that we don't overload our virtual machine with 40GB of data all at once. 
I did this with 

`query + "\n limit 100 "` or even `query + "\n limit 100 offset 100"`

In [17]:
import google.datalab.storage as storage
import google.datalab.bigquery as bq
BigData = bq.Query(query + "\n limit 100").execute().result().to_dataframe()
BigData.head()

,Questiontitle,QuestionBody,QuestionCommentCount,QuestionScore,QuestionTags,AnswerBody,AnswerCommentCount,AnswerScore,NumberOfBadges,NumberOfCommentsMade,...,CodedAnswers,Reputation,UpVotes,DownVotes,ProfileViews,HasWebsite,AllTagsAnswered,IsItUseful,IsItAccepted,UsefulOrAccepted
0,Django on Digital Ocean,<p>So I am beginner Django Developer and have ...,2,0,django|digital-ocean,<p>I have been using Digital Ocean/DO for my D...,0,0,6,1,...,0,79,9,0,9,False,[ssh|digital-ocean],False,False,False
1,How to find which side of a collider has been ...,"<p>In Unity3d, i can get the normal of the sur...",2,2,collision-detection|unity3d|game-physics,"<p>This works:</p>\n\n<pre class=""lang-c# pret...",0,0,20,31,...,12,593,19,0,38,False,"[android|key|android-edittext, android|android...",False,False,False
2,"How do you disable ""Swipe down to close"" on th...","<p>I'm using the core <a href=""https://faceboo...",3,3,react-native|react-navigation,<p>Struggled with it a bit too. Here is what w...,0,0,5,5,...,8,151,48,0,11,False,"[reactjs|react-router, ios|reactjs|react-nativ...",False,False,False
3,Recursive loop to create a family tree? (PHP/M...,"<p>Currently creating my own breeding website,...",1,0,php|html|mysql|arrays|loops,<p>Another starting point for a possible solut...,3,-1,12,26,...,11,597,68,0,16,False,"[javascript|html|google-visualization, html|ht...",False,False,False
4,SSRS 2008 doesn't render report,<p>I have several computers that can render a ...,0,4,reporting-services|ssrs-2008|reporting,<p>If you are using the Server Reporting Servi...,0,0,5,1,...,1,83,9,0,3,False,[windows|android-studio-3.0|android-avd|avd-ma...,False,False,False


### Okay so we can query data and turn it into a dataframe. We can process this in a batchwise fashion later on. Let's try to run some analysis to figure out some statistics on the answers. 

Let's change out the query to get some summary statistics like what percentage of answers are useful and the percentage of answers  become the accepted answer. We'll use a different method to write out a query. This involves the `%%bq`

In [18]:
%%bq query --name SummaryStats
select
countif(Questions.accepted_answer_id=Answers.id)/count(*) as AcceptedAnswersRate, countif(Answers.score>=1)/count(*) as UsefulAnswersRate, 
countif(Answers.score>=1 or Questions.accepted_answer_id=Answers.id)/count(*) as AcceptedOrUsefulRate
from
`bigquery-public-data.stackoverflow.posts_questions` as Questions
join
`bigquery-public-data.stackoverflow.posts_answers` as Answers
on Questions.id = Answers.parent_id
where Questions.score >= 0

In [19]:
SummaryStats.execute().result().to_dataframe()

,AcceptedAnswersRate,UsefulAnswersRate,AcceptedOrUsefulRate
0,0.347271,0.60214,0.675804


<br>
## Analysis

So 60% of the answers get an upvote or more. I'm assuming that upvoted answers are useful. 
Only 35% of answers actually get accepted by the original poster.
If we look at usefulness OR acccepted answers, then around 68% of answers can be considered as 'good contributions'
<br><br>

## Problem

The answers posted are not always useful or acceptable. We can maximize the number of 'good contributions'

Currently, the system relies on user input to judge whether or not an answer is voted up or accepted to be the final answer. 
People like the average programmer are always looking at the answers section of Stackoverflow, and they have to filter through the 35% of the answers that are not voted useful. If no votes have been cast yet, it gets even harder to figure things out. 
<br><br>

## Approach
Can we predict if a certain user can make a  'good contribution' to a question given that the question has not been voted down?
We can direct questions to users that are already known to make good contributions, so that we minimize the wrong answers to a question. 
<br>
<br>
## Today's deliverables were:
1. Google BigQuery API  usage on datalabe
2. Controlling BigQuery output using limits
3. SQL query framework for data on questions, answers and answerer statistics.
<br><br>

## Coming up next. Modelling! 
I'm trying to predict whether a user can make a good contribution to a question based on the question itself and the stats on the user.
This will involve getting the nouns out of the question to add to the keywords, amounts of code in both questions and answers
### More on this, next time!
